## Mini OSS Insight Workshop

👏 欢迎来到 Mini OSS Insight 的 Workshop!

这是关于 Workshop 的完整指南，你可以通过阅读该指南一步一步地搭建起一个 Mini 版的 OSS Insight，并在过程掌握一些使用 TiDB / TiDB Cloud 进行数据分析的技巧。

### 准备

在开始之前，你需要确保在你的开发环境中已经安装了以下软件/工具：

- 可供连接的 TiDB 集群 - 用于存储数据的数据库
- MyCLI - 连接到 TiDB 集群
- Node.js 18.x and above - API Server 的运行时环境
- PNPM - Node.js 的包管理器
- Python 3.x and pip - Jupyter Notebook 需要的运行时环境

#### 1. 启动 TiDB 集群

首先, 你需要开启一个 TiDB 集群。你可以通过查看文档 [TiDB Cloud Quick Start](https://docs.pingcap.com/tidbcloud/tidb-cloud-quickstart#step-1-create-a-tidb-cluster) 学习如何创建一个 **Serverless tier** 集群，这可能会需要 20～25s 的时间。

在你创建完集群，你可以进入到集群的详情页。在这个页面，你可以在 **Connection** 面板中找到数据库集群的连接信息。

<center>
  <img align="middle" width="800" alt="Serverless Tier Cluster Manage Interface" src="https://user-images.githubusercontent.com/5086433/204476069-0ddbdf6f-419c-4291-b929-ccfbd2f5ea5f.png">
  <p><i>Serverless Tier 集群管理界面</i></p>
</center>

You can enter to the cluster's security settings window by clicking on the modify menu in the upper right corner of the cluster details page and generate a root user password.

<center>
  <img width="480" alt="The Cluster Modify Menu" src="https://user-images.githubusercontent.com/85985765/204876779-3a4c6ac4-8814-47cd-b82a-40eb5e4d8f96.png">
  <p><i>The Cluster Modify Menu</i></p>
</center>

<center>
  <img width="720" alt="Security Settings" src="https://user-images.githubusercontent.com/85985765/204877348-5c3e9012-f7bf-42e9-8a03-fd9f14bfc826.png">
  <p><i>Security Settings</i></p>
</center>

#### 3. 个人 GitHub 的 Access Token

You need to prepare a personal access token to allow the application to access the data through the GitHub API.

> **Note**
>
> If you are viewing this document from GitHub codespace, you can skip this step because `GITHUB_TOKEN` is set by default in the codespace environment.
>  

You can learn how to generate one by reading: [Creating a personal access token](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token). Or just click this [link](https://github.com/settings/personal-access-tokens/new) to generate your personal access token quickly.

<center>
  <img align="middle" width="800" alt="Create a New GitHub Personal Access Token" src="https://user-images.githubusercontent.com/5086433/204564273-93cccbe4-d10a-4d1b-a9d1-112a1144712a.png">
  <p><i>Create a new GitHub personal access token</i></p>
</center>


#### 4. 设置环境变量


In [14]:
import getpass
import os

api_server_dot_env="./packages/api-server/.env"
if os.path.exists(api_server_dot_env): 
    con = input(api_server_dot_env + " file already exists, do you want to overwrite it? (y/n) [y]")
    if con != 'y':
        exit();

# Config GitHub personal access token.
github_token = ""
if os.getenv('GITHUB_TOKEN') is None:
    github_token = getpass.getpass(prompt='Enter your personal access token of GitHub: ')
else:
    github_token = os.getenv('GITHUB_TOKEN')

# Config database connection.
db_endpoint = input("The endpoint of TiDB cluster: ")
db_port = input("The port of TiDB cluster [4000]: ")

if db_port == "":
    db_port = "4000"

db_username = input("The username of TiDB cluster: ")
db_password = getpass.getpass(prompt='Enter the password of TiDB cluster: ')
db_name = "ossinsight"
db_enable_ssl = input("Enable ssl connection to the TiDB cluster? (y/n) [y]") or "y"
db_ssl_config='&ssl={"minVersion":"TLSv1.2"}'
if db_enable_ssl == "n":
    db_ssl_config = ""

api_database_url = "mysql://{}:{}@{}:{}/{}?connectionLimit=100&queueLimit=10000{}\n".format(
    db_username, db_password, db_endpoint, db_port, db_name, db_ssl_config
)

etl_database_url = "tidb://{}:{}@{}:{}/{}".format(
    db_username, db_password, db_endpoint, db_port, db_name
)

os.system("gh secret set DB_ENDPOINT --body \"" + db_endpoint + "\" -a codespaces --repo ${GITHUB_REPOSITORY}")
os.system("gh secret set DB_PORT --body \"" + db_port + "\" -a codespaces --repo ${GITHUB_REPOSITORY}")
os.system("gh secret set DB_USERNAME --body \"" + db_username + "\" -a codespaces --repo ${GITHUB_REPOSITORY}")
os.system("gh secret set DB_PASSWORD --body \"" + db_password + "\" -a codespaces --repo ${GITHUB_REPOSITORY}")
os.system("gh secret set DB_NAME --body \"" + db_name + "\" -a codespaces --repo ${GITHUB_REPOSITORY}")
os.system("gh secret set DB_ENABLE_SSL --body \"" + db_enable_ssl + "\" -a codespaces --repo ${GITHUB_REPOSITORY}")
os.system("gh secret set API_DATABASE_URL --body \"" + api_database_url + "\" -a codespaces --repo ${GITHUB_REPOSITORY}")
os.system("gh secret set ETL_DATABASE_URL --body \"" + etl_database_url + "\" -a codespaces --repo ${GITHUB_REPOSITORY}")

# Write to ./packages/api-server/.env file.
with open(api_server_dot_env, "w") as file:
    file.write(
        "DATABASE_URL={}".format(api_database_url)
    )
    file.write("ENABLE_CACHE=false\n")
    file.write("GITHUB_ACCESS_TOKENS={}\n".format(github_token))

print("Setup successfully!")

#### 5. 验证是否能够连接到 TiDB 集群

上执行下面的 SQL 语句验证是否能够连接到 TiDB 集群:

In [1]:
!mycli -h ${DB_ENDPOINT} -P ${DB_PORT} -u ${DB_USERNAME} -p ${DB_PASSWORD} -D ${DB_NAME} \
    --ssl-ca=/etc/ssl/certs/ca-certificates.crt \
    --ssl-verify-server-cert \
     -e 'SELECT tidb_version()\G'

tidb_version()
Release Version: v6.3.0-serverless
Edition: Community
Git Commit Hash: e87c16b215d518aed4921b8ef3b13e90e3ed6e2d
Git Branch: release-6.3-serverless
UTC Build Time: 2022-11-25 09:31:28
GoVersion: go1.19
Race Enabled: false
TiKV Min Version: 6.1.0
Check Table Before Drop: false
Store: tikv


如果成功执行，你会看到 TiDB 集群的版本信息。

### 加载数据

从 2011 年开始，GitHub 便对外提供了一个 [`/events`](https://api.github.com/events) API，我们可以通过这个 API 来获取在 GitHub 上发生的近乎实时（GitHub 设定了 5 分钟的延迟）的公开事件。

> **GitHub API 文档**
> 
> Link: https://docs.github.com/en/rest/activity/events
>

In [11]:
!curl -s \
  -H "Accept: application/vnd.github.v3+json" \
  -H "Authorization: token $GITHUB_TOKEN" \
  https://api.github.com/events

[
  {
    "id": "25618713222",
    "type": "PushEvent",
    "actor": {
      "id": 80751634,
      "login": "vacseeker",
      "display_login": "vacseeker",
      "gravatar_id": "",
      "url": "https://api.github.com/users/vacseeker",
      "avatar_url": "https://avatars.githubusercontent.com/u/80751634?"
    },
    "repo": {
      "id": 348289466,
      "name": "vacseeker/vacseeker.github.io",
      "url": "https://api.github.com/repos/vacseeker/vacseeker.github.io"
    },
    "payload": {
      "push_id": 11862047375,
      "size": 1,
      "distinct_size": 1,
      "ref": "refs/heads/main",
      "head": "99ad4573b8edc52b2535ef196ebdae751b2a10b2",
      "before": "80118a6f474389f0c777132f3e81f60e4594b26c",
      "commits": [
        {
          "sha": "99ad4573b8edc52b2535ef196ebdae751b2a10b2",
          "author": {
            "email": "80751634+vacseeker@users.noreply.github.com",
            "name": "vacseeker"
          },
          "message": "snred_commit_message",
         

OSS Insight 已经收集了超过 53 亿条公开事件数据，你可以通过访问 API 或在网站上查看当前收集的事件总数：

In [2]:
!curl -s https://api.ossinsight.io/q/events-total | jq -r ".data[0]"

{
  "cnt": 5322990364,
  "latest_created_at": "2022-12-02T14:53:58.000Z",
  "latest_timestamp": 1669992838
}


#### 导入实时数据

通过 [GitHub 的 API 文档](https://docs.github.com/en/rest/activity/events), 我们知道可以通过请求 `/events` API 来获取准实时的 events 数据。你可以执行下面的命令查看这个接口返回数据的结构：

In [ ]:
!docker-compose pull
!docker-compose up -d etl

#### 导入历史数据

由于通过 `/events` API 我们无法完全获取到所有的历史数据, 在 OSS Insight 的数据准备阶段，我们利用了 [GH Archive](https://gharchive.org) 上归档数据，将其导入数据库当中.

| Query | Downloadable Files |
| ---- | ---- |
| Activity for 1/1/2015 @ 3PM UTC |	`https://data.gharchive.org/2015-01-01-15.json.gz` |
| Activity for 1/1/2015 | `https://data.gharchive.org/2015-01-01-{0..23}.json.gz` |
| Activity for all of January 2015 | `https://data.gharchive.org/2015-01-{01..31}-{0..23}.json.gz` |

为了方便演示我们在 AWS S3 上保存了一份示例数据，你可以使用 TiDB Cloud 的 Import 功能将其导入数据库当中：

S3 URI:

```
s3://ossinsight/workshop/oss_database_sample/
```

Role ARN:

```
arn:aws:iam::494090988690:role/tidb-cloud-reader
```

## 一些背景知识

#### 优化器

SQL 是一种声明式语言，而非过程性的语言。也就是说，它描述的是最终结果应该如何，而非按顺序执行的步骤。因此在 TiDB 在处理 SQL 语句的过程中，除了要确保能够正确返回语句所描述的最终结果，还要确定 SQL 语句的执行过程，并且尽可能的对 SQL 语句进行性能优化，从而得到最佳的执行计划。

SQL 性能优化的过程，可以理解为 GPS 导航的过程。你提供地址后，GPS 软件利用各种统计信息（例如以前的行程、速度限制等元数据，以及实时交通信息）规划出一条最省时的路线。这与 TiDB 中的 SQL 性能优化过程相对应。

我们通常会把负责这部分 SQL 优化工作的组件成为**优化器（Optimizer）**。

如下图所示，一条 SQL 语句在经过语法解析和校验之后，经过优化器的逻辑优化和物理优化来制定最终的执行计划，交给执行器（Executor）去执行。和 GPS 软件一样，优化器在进行执行计划规划的时候也会利用到一些统计信息来进行评估。

<center>
  <img width="640" alt="The Cluster Modify Menu" src="https://user-images.githubusercontent.com/85985765/205136445-3d9dd676-56bc-4f75-bf6d-cdb1af8c1b79.png">
  <p><i>SQL 优化流程图</i></p>
</center>

上图其实是一个简化的示意图，因为它只描述了以 TiKV 为存储引擎的情况。但是在 TiDB 支持 HTAP 能力后，引进了负责分析负载的 TiFlash 存储引擎。这给优化器带来了新的挑战，优化器需要在制定执行计划和进行物理
优化时需要考虑：

> 这条 SQL 应该使用哪个存储引擎执行性能会更好，选 TiKV 还是 TiFlash？

#### Optimizer Hints

> 如果优化器难以抉择，那我们不妨给它一点提示，告诉它我们希望使用哪种存储引擎执行?

TiDB 的优化器提供了 `READ_FROM_STORAGE` Hint 来允许开发者在 SQL 语句级别来控制哪一张表应该使用哪个执行引擎去执行。例如，下在面这条 SQL 语句当中，TiDB 从 TiFlash 上读取 t1 表的数据，在 TiKV 上读取 t2 的数据。

```sql
SELECT /*+ READ_FROM_STORAGE(TIFLASH[t1], TIKV[t2]) */ t1.a FROM t t1, t t2 WHERE t1.a = t2.a;
```

#### 代价模型

> 使用 Optimizer Hints 很简单，但是有没有更智能的方法，特别是在做临时查询时，优化器能否智能的选择应该使用哪种引擎查询效率最高？

<center>
  <img width="640" alt="物理优化中的代码模型" src="https://user-images.githubusercontent.com/85985765/205192450-c23f2199-1ab1-41d2-90f3-c17c5d6e252e.png">
  <p><i>SQL 优化流程图</i></p>
</center>

TiDB v6.2.0 引入了新的代价模型 Cost Model Version 2。

Cost Model Version 2 对代价公式进行了更精确的回归校准，调整了部分代价公式，比此前版本的代价公式更加准确。

OSS Insight 使用了该版本的 Cost Model, 但是因为目前 Cost Model Version 2 还处于实验阶段，需要通过执行以下 SQL 语句进行启用：

In [ ]:
set global tidb_enable_new_cost_interface = 'ON';
set global tidb_cost_model_version = 2;

在后续版本当中，Cost Model Version 2 会替代掉原有的 Cost Model 作为默认的代码模型。

## Demo SQL

下面，我们会以 OSS Insight 为示例，讲解如何使用 TiDB 进行数据分析以及在数据分析时如何更好的通过优化器来发挥行存 TiKV 和列存 TiFlash 各自的优势。


#### 定义 Query 工具函数

首先，请执行下面的 Python 代码，初始化一个 query 函数用于后续的 SQL 查询：


In [4]:
import mysql.connector as connection
import pandas as pd

db_host = os.getenv('DB_ENDPOINT')
db_port = os.getenv("DB_PORT")
db_username = os.getenv('DB_USERNAME')
db_password = os.getenv('DB_PASSWORD')
db_name = os.getenv('DB_NAME')

mydb = connection.connect(
  host=db_host,
  port=db_port,
  user=db_username,
  password=db_password,
  database=db_name
)

def execute(sql):
  try:
    df = pd.read_sql(sql, mydb)
    return df.style.set_properties(**{'text-align': 'left'})
  except Exception as e:
    print(str(e))

def query(sql):
  try:

    df = pd.read_sql(sql, mydb)
    return df.style.set_properties(**{'text-align': 'left'})
  except Exception as e:
    print(str(e))

#### Example 1

熟悉 TiDB 的用户可能会知道，在 TiDB 的版本规则里会存在两种版本：

- 一种是为了能够更快交付迭代而设计的开发里程碑版本 (Development Milestone Releases, DMR)
- 另外一种是提供长期技术支持的长期支持版本 (Long-Term Support Releases, LTS)

在 LTS 生命周期内会按需发布补丁版本 (Patch Release)。如果一个修复 BUG 的 Pull Request 会被 Cherry-Pick 到受影响的 LTS 分支进行修复，这部分工作会由机器人根据研发和 QA 团队确定的受影响版本列表进行自动的 Cherry—Pick，所以你会在 TiDB 的代码仓库当中看见不少由机器人发起的 Pull Request。

> 🤔 那么，TiDB 的社区机器人（`ti-chi-bot`）一共创建了多少个 Pull Request 呢？

我们可以通过下面的 SQL 语句进行查询：

In [14]:
query("""
SELECT repo_name, COUNT(1) AS prs
FROM github_events
WHERE
    type = 'PullRequestEvent'
    AND action = 'opened'
    AND actor_login = 'ti-chi-bot'
GROUP BY repo_name
""")

,repo_name,prs
0,pingcap/tidb,138


通过 `EXPLAIN ANALYZE` 查看上面 SQL 语句的执行计划：

In [15]:
query("""
EXPLAIN ANALYZE SELECT repo_name, COUNT(1) AS prs
FROM github_events
WHERE
    type = 'PullRequestEvent'
    AND action = 'opened'
    AND actor_login = 'ti-chi-bot'
GROUP BY repo_name
""")

,id,estRows,actRows,task,access object,execution info,operator info,memory,disk
0,Projection_5,1.00,1,root,,"time:16.9ms, loops:2, Concurrency:OFF","ossinsight.github_events.repo_name, Column#34",1016 Bytes,N/A
1,└─HashAgg_20,1.00,1,root,,"time:16.9ms, loops:2, partial_worker:{wall_time:16.866894ms, concurrency:5, task_num:1, tot_wait:84.067782ms, tot_exec:25.378µs, tot_time:84.102555ms, max:16.828683ms, p95:16.828683ms}, final_worker:{wall_time:16.918265ms, concurrency:5, task_num:1, tot_wait:84.272723ms, tot_exec:29.637µs, tot_time:84.306145ms, max:16.878693ms, p95:16.878693ms}","group by:ossinsight.github_events.repo_name, funcs:count(Column#38)->Column#34, funcs:firstrow(ossinsight.github_events.repo_name)->ossinsight.github_events.repo_name",16.1 KB,N/A
2,└─IndexLookUp_21,1.00,1,root,partition:pull_request_event,"time:16.8ms, loops:2, index_task: {total_time: 3.36ms, fetch_handle: 3.34ms, build: 2.66µs, wait: 19.2µs}, table_task: {total_time: 39.2ms, num: 3, concurrency: 5}",,131.5 KB,N/A
3,├─IndexRangeScan_17(Build),12145.00,6563,cop[tikv],"table:github_events, index:index_github_events_on_actor_login(actor_login)","time:3.05ms, loops:9, cop_task: {num: 1, max: 2.95ms, proc_keys: 6564, tot_proc: 4ms, rpc_num: 1, rpc_time: 2.92ms, copr_cache_hit_ratio: 0.00, distsql_concurrency: 15}, tikv_task:{time:4ms, loops:11}, scan_detail: {total_process_keys: 6564, total_process_keys_size: 466024, total_keys: 6565, rocksdb: {block: {}}}","range:[""ti-chi-bot"",""ti-chi-bot""], keep order:false",N/A,N/A
4,└─HashAgg_7(Probe),1.00,1,cop[tikv],,"time:36.8ms, loops:4, cop_task: {num: 3, max: 12.1ms, min: 11.5ms, avg: 11.8ms, p95: 12.1ms, max_proc_keys: 2499, p95_proc_keys: 2499, tot_proc: 24ms, rpc_num: 3, rpc_time: 35.4ms, copr_cache_hit_ratio: 0.00, distsql_concurrency: 15}, tikv_task:{proc max:8ms, min:8ms, avg: 8ms, p80:8ms, p95:8ms, iters:8, tasks:3}, scan_detail: {total_process_keys: 6563, total_process_keys_size: 1881239, total_keys: 6728, rocksdb: {block: {}}}","group by:ossinsight.github_events.repo_name, funcs:count(1)->Column#38",N/A,N/A
5,└─Selection_19,616.14,138,cop[tikv],,"tikv_task:{proc max:8ms, min:8ms, avg: 8ms, p80:8ms, p95:8ms, iters:8, tasks:3}","eq(ossinsight.github_events.action, ""opened""), eq(ossinsight.github_events.type, ""PullRequestEvent"")",N/A,N/A
6,└─TableRowIDScan_18,12145.00,6563,cop[tikv],table:github_events,"tikv_task:{proc max:8ms, min:8ms, avg: 8ms, p80:8ms, p95:8ms, iters:8, tasks:3}",keep order:false,N/A,N/A


通过执行计划，我们可以知道这个 SQL 会使用 TiKV 存储引擎来读取 `github_events` 表中的数据，并且使用了索引 `index_github_events_on_actor_login`。

#### Example 2

> 🤔 除了通过聚合函数来进行简单数据分析，TiDB 是否可以做更复杂的数据分析吗？

比如我们想要知道某个指标的增长趋势，以代码仓库 `pingcap/tidb` 为例，我们能否通过 SQL 查询该仓库 Star 的增长趋势 (按月累计)，这样的 SQL 语句我们会怎么写?

In [22]:
query("""
  SELECT
    actor_id,
    actor_login,
    repo_id,
    repo_name,
    created_at,
    DATE_FORMAT(created_at, '%Y-%m-01') as event_month
  FROM
    github_events
  WHERE
    type = 'WatchEvent'
    AND repo_name = 'pingcap/tidb'
  ORDER BY created_at
  LIMIT 10
""")

,actor_id,actor_login,repo_id,repo_name,created_at,event_month
0,1953644,qiuyesuifeng,41986369,pingcap/tidb,2015-09-06 04:15:40,2015-09-01
1,878009,ngaut,41986369,pingcap/tidb,2015-09-06 04:15:44,2015-09-01
2,1192573,shenli,41986369,pingcap/tidb,2015-09-06 04:15:53,2015-09-01
3,4242506,zimulala,41986369,pingcap/tidb,2015-09-06 04:15:53,2015-09-01
4,891222,coocood,41986369,pingcap/tidb,2015-09-06 04:16:19,2015-09-01
5,541236,tomzhang,41986369,pingcap/tidb,2015-09-06 04:17:26,2015-09-01
6,1782276,FlamingTree,41986369,pingcap/tidb,2015-09-06 04:17:45,2015-09-01
7,14066328,nengwang,41986369,pingcap/tidb,2015-09-06 04:18:10,2015-09-01
8,1468284,se77en,41986369,pingcap/tidb,2015-09-06 04:19:55,2015-09-01
9,773853,c4pt0r,41986369,pingcap/tidb,2015-09-06 04:20:56,2015-09-01


我们可以利用[窗口函数](https://docs.pingcap.com/zh/tidb/stable/window-functions)来实现这一个需求。窗口函数通常也被称为 “OLAP 函数”，又或者是 “开窗函数”。

例如，在下面的 SQL 语句当中，我们通过 `OVER` 语句定义窗口的范围，TiDB 会将数据行按照 `created_at` 列进行排序，然后对这些数据行按顺序扫描，将当前行之前的所有行作为一个窗口。

随着扫描的行数越多，窗口也会逐步扩大。对于每个窗口都会执行一次聚合函数 `COUNT(1)` 来统计窗口内的事件总数，从而可以得到一组随着时间累积增长的统计数据。

In [7]:
query("""
SELECT
    created_at,
    COUNT(1) OVER (ORDER BY created_at) AS stars
FROM
    github_events
WHERE
    type = 'WatchEvent'
    AND repo_name = 'pingcap/tidb'
ORDER BY created_at
LIMIT 100;
""");

在这个基础之上，我们进行一个按月聚合，使用 `DATE_FORMAT('%Y-%m-01')` 函数将同一个月的事件记录进行分组统计。

In [8]:
query("""
SELECT
  event_month, stars
FROM (
  SELECT
    DATE_FORMAT(created_at, '%Y-%m-01') as event_month,
    COUNT(1) OVER (ORDER BY DATE_FORMAT(created_at, '%Y-%m-01')) AS stars,
    ROW_NUMBER() OVER (PARTITION BY DATE_FORMAT(created_at, '%Y-%m-01')) AS row_num
  FROM
    github_events
  WHERE
    type = 'WatchEvent'
    AND repo_name = 'pingcap/tidb'
  ORDER BY 1
) sub
WHERE row_num = 1
ORDER BY event_month DESC
""");

通过 `EXPLAIN` 语句查看其执行计划：

In [15]:
query("""
EXPLAIN ANALYZE SELECT
  event_month, stars
FROM (
  SELECT
    DATE_FORMAT(created_at, '%Y-%m-01') as event_month,
    COUNT(1) OVER (ORDER BY DATE_FORMAT(created_at, '%Y-%m-01')) AS stars,
    ROW_NUMBER() OVER (PARTITION BY DATE_FORMAT(created_at, '%Y-%m-01')) AS row_num
  FROM
    github_events ge
  WHERE
    type = 'WatchEvent'
    AND repo_name = 'pingcap/tidb'
  ORDER BY 1
) sub
WHERE row_num = 1
ORDER BY event_month DESC;
""");

我们发现对于 OLAP 函数，TiDB 的优化器仍然会使用 TiKV 作为存储引擎进行数据查询。

这是因为在默认情况下，TiDB 不会是为 `github_events` 表创建 TiFlash 副本的，你需要手动地进行创建，创建副本的方法也十分简单，你只需要执行下面这条 SQL 语句，就可以为 `github_events` 表创建一个 TiFlash 副本：

In [ ]:
query("""
ALTER TABLE ossinsight.github_events SET TIFLASH REPLICA 1;
""")

执行完上面的 SQL 语句后，TiFlash 会在后台自动地完成副本的创建，你可以通过执行下面的 SQL 语句来查看副本的创建进度：

In [12]:
query("""
SELECT * FROM information_schema.tiflash_replica WHERE TABLE_SCHEMA = 'ossinsight' and TABLE_NAME = 'github_events';
""")

,TABLE_SCHEMA,TABLE_NAME,TABLE_ID,REPLICA_COUNT,LOCATION_LABELS,AVAILABLE,PROGRESS,TABLE_MODE
0,ossinsight,github_events,100,1,,1,1.000000,NORMAL


当 “PROCESS” 为 1 时，表示 TiFlash 副本已经创建完成，再次执行上面的分析查询语句：

In [14]:
query("""
EXPLAIN ANALYZE SELECT
  event_month, stars
FROM (
  SELECT
    DATE_FORMAT(created_at, '%Y-%m-01') as event_month,
    COUNT(1) OVER (ORDER BY DATE_FORMAT(created_at, '%Y-%m-01')) AS stars,
    ROW_NUMBER() OVER (PARTITION BY DATE_FORMAT(created_at, '%Y-%m-01')) AS row_num
  FROM
    github_events ge
  WHERE
    type = 'WatchEvent'
    AND repo_name = 'pingcap/tidb'
  ORDER BY 1
) sub
WHERE row_num = 1
ORDER BY event_month DESC;
""");

我们会发现此时的执行计划当中已经出现了 `cop[tiflash]` 的字样，说明此时的查询已经在使用 TiFlash 存储引擎来读取数据。

> 🤔 但是看上去前后查询的速度并没有多大的差异。

#### Example 3

我们来看一下更加复杂的例子，在前面的查询当中，我们只是在查询一个仓库的数据，TiDB 也许还能够借助 TiKV 走索引的方式找到一条更高效的执行计划。

但是有的时候，我们的数据分析可能并没有这么简单，可能需要同时分析 N 个代码仓库，甚至所有代码仓库的数据。又或者是我们希望能够进行大批量的数据报表统计，这个时候需要扫描更为大量的数据，这时候优化器即便仍然选择了索引查询的方式，查询的效率可能仍然不会很高。

首先，我们应用我们在背景知识当中学习到的技巧，通过 `/*+ READ_FROM_STORAGE(TIKV[ge]) */` Hints 来告诉优化器在查询 `github_events` 表时强制使用 TiKV 存储引擎。

In [17]:
query("""
WITH bots_with_first_seen AS (
    SELECT
        /*+ READ_FROM_STORAGE(TIKV[ge]) */
        actor_login, MIN(YEAR(created_at)) AS first_seen_at
    FROM github_events ge
    WHERE
        actor_login REGEXP '^(bot-.+|.+bot|.+\\[bot\\]|.+-bot-.+|robot-.+|.+-ci-.+|.+-ci|.+-testing|.+clabot.+|.+-gerrit|k8s-.+|.+-machine|.+-automation|github-.+|.+-github|.+-service|.+-builds|codecov-.+|.+teamcity.+|jenkins-.+|.+-jira-.+)$'
    GROUP BY actor_login
    ORDER BY first_seen_at
),  acc AS (
    SELECT
        COUNT(actor_login) OVER (ORDER BY first_seen_at) AS cnt,
        first_seen_at AS event_year
    FROM
        bots_with_first_seen AS bwfs
    ORDER BY event_year
)
SELECT ANY_VALUE(cnt) AS bots_total, event_year
FROM acc
GROUP BY event_year
ORDER BY event_year;
""");

记录下查询的时间。

然后我们通过 `/*+ READ_FROM_STORAGE(TIFLASH[ge]) */` Hints 来强制优化器使用 TiFlash 存储引擎来读取 `github_events` 表，再次执行：

In [18]:
query("""
WITH bots_with_first_seen AS (
    SELECT
        /*+ READ_FROM_STORAGE(TIFLASH[ge]) */
        actor_login, MIN(YEAR(created_at)) AS first_seen_at
    FROM github_events ge
    WHERE
        actor_login REGEXP '^(bot-.+|.+bot|.+\\[bot\\]|.+-bot-.+|robot-.+|.+-ci-.+|.+-ci|.+-testing|.+clabot.+|.+-gerrit|k8s-.+|.+-machine|.+-automation|github-.+|.+-github|.+-service|.+-builds|codecov-.+|.+teamcity.+|jenkins-.+|.+-jira-.+)$'
    GROUP BY actor_login
    ORDER BY first_seen_at
),  acc AS (
    SELECT
        COUNT(actor_login) OVER (ORDER BY first_seen_at) AS cnt,
        first_seen_at AS event_year
    FROM
        bots_with_first_seen AS bwfs
    ORDER BY event_year
)
SELECT ANY_VALUE(cnt) AS bots_total, event_year
FROM acc
GROUP BY event_year
ORDER BY event_year;
""");

#### Example 4

In [ ]:
> 🤔